In [ ]:
!pip install --upgrade pip
!pip install numpy
!pip install pandas
!pip install xlrd

In [2]:
import numpy as np
import pandas
pandas.set_option('display.max_colwidth', -1)

metadata_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/variables_clara.csv', low_memory=False, index_col=0, encoding='utf-8')
geo_metadata_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geo_variables_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')
print('Read metadata...')

geo_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/geo_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')
print('Read geography...')

data_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/data_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')
print('Read data (estimates)...')

error_df = pandas.read_csv('/home/idies/workspace/raddick_acs_data/error_acs2016_clara.csv', low_memory=False, index_col=0, encoding='utf-8')
print('Read margins of error...')

print('Done')

Read metadata...
Read geography...
Read data (estimates)...
Read margins of error...
Done


In [ ]:
#metadata_df
#metadata_df[metadata_df['sequence_number'] == 150]
#metadata_df
#B26001_001
#GROUP QUARTERS POPULATION for Population In Group Quarters% Total

#string1 = 'Baltimore'
#string2 = ''
#string3 = ''

#geovarlist = metadata_df['variable'][metadata_df['description'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x))].values.tolist()
#geo_df[geo_df['NAME'].apply(lambda x: (string1 in x))]

#print('{0:}'.format(geo_df['NAME'].loc['05000US24510']))
#print('{0:} {1:,.0f}'.format(metadata_df['description'].loc['B26001_001'], data_df['B26001_001'].loc['05000US24510']))

#data_df['B26001_001'].loc['05000US24510']
#data_df      
#05000US24510
#data_df['B26001_001'].sample(1)

geolist = geo_df['GEOID'].sample(3).values.tolist()
#geolist.append('afwwfe')
    
thelist = ['B01001_001', 'B26001_001']
#thelist.append('xxx')
#thelist  = []
ourdata_df = get_data(data_df, thelist, geolist)
#ourdata_df

phrases = []
phrases.append('Total Population')
phrases.append('AGE')
phrases.append('Under 5')
#phrases.append('Male')

thesevars = find_variables4(metadata_df, phrases)
#find_variables(metadata_df, phrases)
metadata_df[metadata_df['variable'].apply(lambda x: x in thesevars)]
#print(thesevars)

In [ ]:
def find_variables4(mdf, tofindlist):
    # NEED TO REWRITE TO USE REGEX INSTEAD, because now only "Male" returns men; "male" returns women b/c feMALE
    varlist = []
    if (len(tofindlist) == 0):
        print('CAUTION: No search phrases specified, returning every variable!')
    if (len(tofindlist) > 4):
        print('ERROR: list contains more than four search phrases, searching for only the first three')
    try:
        string1 = tofindlist[0]
    except IndexError:
        string1 = ''
    try:
        string2 = tofindlist[1]
    except IndexError:
        string2 = ''
    try:
        string3 = tofindlist[2]
    except IndexError:
        string3 = ''
    try:
        string4 = tofindlist[3]
    except IndexError:
        string4 = ''
            
    varlist = mdf['variable'][mdf['description'].apply(lambda x: (string1 in x) & (string2 in x) & (string3 in x) & (string4 in x))].values.tolist()
    return varlist

def get_data(df, varlist, geolist):
    y_df = get_geography(df, geolist)
    rdf = get_columns(y_df, varlist)
    return rdf

def get_columns(df, varlist):
    if (len(varlist) == 0):
        print('CAUTION: No variables specified')
    rdf = pandas.DataFrame()
    if (len(df) > 0):
        vars_that_exist = ['GEOID']
        for thisvar in varlist:
            if (thisvar in df.columns):
                vars_that_exist.append(thisvar)
            else:
                print('Variable {0:} not found. Skipping.'.format(thisvar))

        rdf = df[vars_that_exist]
        rdf.index.name = ''
    else:
        print('CAUTION: No geopgraphies specified')
    return rdf

def get_geography(df, geolist):
    rdf = pandas.DataFrame()
    if (len(df) > 0):
        for thisgeo in geolist:
            try:
                rdf = rdf.append(df.loc[thisgeo])
            except:
                print('Geography {0:} not found. Skipping.'.format(thisgeo))
    else:
        print('No variables specified')
    return rdf
    
print('Done')

In [8]:
#geo_metadata_df

# SUMLEVEL documented at https://www.census.gov/programs-surveys/acs/geography-acs/geography-boundaries-by-year/2016.html
#010,US,16,"United States -- In metropolitan statistical area"
#020,Region,48,"South Region -- In micropolitan statistical area -- not in principal city"
#030,Division,108,"South Atlantic Division -- Not in metropolitan or micropolitan statistical area"
#040,State or State Equivalent,588,"New York -- In micropolitan statistical area -- not in principal city"
#050,"County or equivalent (Alaska boroughs, Louisiana parishes, Puerto Rico municipios, independent cities)",856,"Miami-Dade County, Florida"
#060,State-County-County Subdivision,248,"Danbury town, Fairfield County, Connecticut"
#067,"Minor Civil Division (MCD)/Barrio and Barrio-Pueblo",0,""
#140,Census Tract,0,""
#150,Census Block Group,0,""
#160,Census Designated Place (CDP)/Zona Urbana and Comunidad,610,"Urban Honolulu CDP, Hawaii"
#170,Consolidated City,0,""
#230,Alaska Native Regional Corporations (ANRCs),3,"Doyon Alaska Native Regional Corporation, Alaska"
#250,Federal or State American Indian Reservation (federal AIR/state AIR) or Hawaiian Home Land (HHL),12
#251,American Indian Tribal Subdivision, federal AIR,0
#252,Federal or State American Indian Reservation (federal AIR/state AIR),0
#254,Federal American Indian Reservation (federal AIR) or Hawaiian Home Land (HHL),0
#256,Tribal Tract,0
#258,Tribal Block Group,0
#310,Metropolitan Statistical Area/Micropolitan Statistical Area,518
#312,Metropolitan Statistical Area/Micropolitan Statistical Area - Principal City,395
#314,Metropolitan Statistical Area-Metropolitan Division,31
#330,Combined Statistical Area,171
#335,Combined New England City and Town Area,7
#337,Combined New England City and Town Area-New England City and Town Area
#350,New England City and Town Area (NECTA),24
#352,New England City and Town Area - Principal City,21
#355,New England City and Town Area (NECTA)-NECTA Division,10
#361 State-New England City and Town Area-Principal City,0
#400,Urban Area,437
#500,"Congressional District, Delegate District DC, Resident Commissioner District PR (115th Congress)",447
#610,State Legislative District (Upper Chamber),0
#620,State Legislative District (Lower Chamber),0
#620,State-State Legislative District (Lower Chamber),0
#700,State-County-Voting District/Remainder,0
#795,Public Use Microdata Area (PUMA),2443
#860,5-digit ZIP Code Tabulation Area (ZCTA),0
# 950 School District (Elementary),78
# 960 School District (Secondary),94
# 970 School District (Unified),877

geo_df[['SUMLEVEL','NAME']][geo_df['SUMLEVEL'] == 250].sample(5)
#geo_df.dtypes

,SUMLEVEL,NAME
25000US2430,250,"Navajo Nation Reservation and Off-Reservation Trust Land, AZ--NM--UT"
25000US5590,250,"Choctaw OTSA, OK"
25000US9815,250,"Lumbee SDTSA, NC"
25000US5580,250,"Chickasaw OTSA, OK"
25000US9550,250,"Cher-O-Creek SDTSA, AL"


In [ ]:
geo_df['NAME'][geo_df.index=='05000US35013']

In [ ]:
states = ['ak', 'al', 'ar', 'az', 'ca', 'co', 'ct', 'dc', 'de', 'fl', 'ga', 'hi', 'ia', 'id', 'il', 'in']
states += ['ks', 'ky', 'la', 'ma', 'md', 'me', 'mi', 'mn', 'mo', 'ms', 'mt', 'nc', 'nd', 'ne', 'nh', 'nj']
states += ['nm', 'nv', 'ny', 'oh', 'ok', 'or', 'pa', 'pr', 'ri', 'sc', 'sd', 'tn', 'tx', 'us', 'ut' ,'va', 'vt', 'wa', 'wi', 'wv', 'wy']

geo_df = pandas.DataFrame()
geo_metadata_df = pandas.DataFrame()

geo_metadata_filename = '/home/idies/workspace/raddick_acs_data/metadata/2016_SFGeoFileTemplate.xls'
geo_metadata_df = pandas.read_excel(geo_metadata_filename, header=None)
geo_metadata_df = geo_metadata_df.dropna(axis=1)
geo_metadata_df.columns = geo_metadata_df.loc[0]
geo_metadata_df = geo_metadata_df.drop(0, axis=0)
geo_metadata_df.index = ['description']

for onestate in states:
    statefilename = '/home/idies/workspace/raddick_acs_data/geography/g20161{0:}.csv'.format(onestate)
#    print(statefilename)
    onestate_df = pandas.read_csv(statefilename, header=None, encoding='utf-8')
    geo_df = geo_df.append(onestate_df)

geo_df.columns = geo_metadata_df.columns
geo_df = geo_df.set_index('GEOID', drop=False)
geo_df.index.name = ''

geo_metadata_df = geo_metadata_df.T
geo_metadata_df = geo_metadata_df.reset_index()
geo_metadata_df.columns = ['variable', 'description']
geo_metadata_df = geo_metadata_df.set_index('variable', drop=False)
geo_metadata_df.index.name = ''

print('Writing geography variables file...')
geo_metadata_df.to_csv('/home/idies/workspace/raddick_acs_data/geo_variables_acs2016_clara.csv', encoding='utf-8')

print('Writing geography file...')
geo_df.to_csv('/home/idies/workspace/raddick_acs_data/geo_acs2016_clara.csv', encoding='utf-8')

print('Done!')